# 如果一个小姐姐不够，那就三个小姐姐一起来

通过deeplabv3p_xception65_humanseg实现一键抠图



## DeepLabv3+ 介绍
 [DeepLabv3+](https://arxiv.org/abs/1802.02611)是DeepLab语义分割系列网络的最新作，其前作有 [DeepLabv1](http://arxiv.org/pdf/1412.7062v3.pdf)，[DeepLabv2](https://arxiv.org/pdf/1606.00915.pdf), [DeepLabv3](https://arxiv.org/abs/1802.02611),
在最新作中，DeepLab的作者通过encoder-decoder进行多尺度信息的融合，同时保留了原来的空洞卷积和ASPP层，
其骨干网络使用了Xception模型，提高了语义分割的健壮性和运行速率，在 PASCAL VOC 2012 dataset取得新的state-of-art performance，89.0mIOU。

在PaddleSeg当前实现中，支持两种backbone

MobileNetV2:
适用于移动设备的快速网络，如果对分割性能有较高的要求，请使用这一backbone网络。

Xception:
DeepLabv3+原始实现的backbone网络，兼顾了精度和性能，适用于服务端部署。

![](https://ai-studio-static-online.cdn.bcebos.com/83acbe5790e24e84b3ea4ea8e181421d9f709aa8803e414986d66a40ae0c5cf5)

#### ASPP (Atrous Spatial Pyramid Pooling)
通过不同采样率的空洞卷积获取不同尺度的特征信息。具体可参见[DeepLabv2](https://arxiv.org/pdf/1606.00915.pdf)
![](https://ai-studio-static-online.cdn.bcebos.com/39495f7947d24f818d2b988481b7f46f0235b6fcd1134bd4a30ebd3bae3d862c)


## 一、设置必要环境


In [1]:
!pip install paddlehub==1.6.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
from PIL import Image
import paddlehub as hub
import cv2
from moviepy.editor import VideoFileClip
import shutil


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"#GPU模式

## 二、抠图
通过deeplabv3p_xception65_humanseg实现一键抠图

In [4]:
#视频转jpg
def video_to_pic():
    if os.path.exists('./video_pic'):
        shutil.rmtree('./video_pic')#删除文件夹以及文件       
    os.mkdir('./video_pic')#创建文件夹
    index = 0
    cap = cv2.VideoCapture('./xxm.mp4')
    ret,frame = cap.read()
    while ret:
        cv2.imwrite('./video_pic/%d.jpg'%index, frame)
        index += 1
        ret,frame = cap.read()
    cap.release()
    print('Video cut finish, all %d frame' % index)
video_to_pic()

Video cut finish, all 308 frame


In [5]:
#图片平移
def tl(img_path):
    '''x轴平移输出两张jpg图片,dest1,dest2'''
    img = cv2.imread(img_path)
    rows,cols, channel = img.shape

# 平移矩阵M：[[1,0,x],[0,1,y]]
    M1 = np.float32([[1,0,400],[0,1,0]])
    M2 = np.float32([[1,0,-400],[0,1,0]])
    dest1 = cv2.warpAffine(img,M1,(cols,rows), borderValue=(255, 255, 255))#仿射平移
    dest2 = cv2.warpAffine(img,M2,(cols,rows), borderValue=(255, 255, 255))
    
    cv2.imwrite('./dest1'+'/'+file,dest1)#保存图片
    cv2.imwrite('./dest2'+'/'+file,dest2)
    # print('保存平移图片成功')

In [6]:
#保存平移后的图片
filename_list = os.listdir('video_pic')
if os.path.exists('dest1'):
    shutil.rmtree('dest1')
os.mkdir('dest1')
if os.path.exists('dest2'):
    shutil.rmtree('dest2')
os.mkdir('dest2')
for file in filename_list:
    tl('./video_pic'+'/'+file)
print("文件已创建")

文件已创建


In [7]:
#扣出平移后图片中的人像
def seg_pic(path,savepath):
    i = 0
    module= hub.Module(name="deeplabv3p_xception65_humanseg")
    pic_path = path

    if os.path.exists(savepath):
        shutil.rmtree(savepath)
    os.mkdir(savepath)
    
    filename_list = os.listdir(pic_path)
    filename_list.sort(key=lambda x: int(x[:-4]))#图片排序
    
    for fname in filename_list:
        result = module.segmentation(images=[cv2.imread(pic_path+'/'+fname)], visualization=True, output_dir=savepath,use_gpu=True)
        #print('已处理处理',pic_path,'图片：', i)
        i+=1
    
    filename_list = os.listdir(savepath)
    filename_list.sort()
    i=0
    for file in filename_list:
        os.rename(savepath+'/'+file,savepath+'/'+str(i)+'.jpg')#（修改前目录名，修改后名）
        i+=1
    print('抠图完成')
#左边原图路径，右边抠图输出路径
seg_pic('dest1','res_dest1')
seg_pic('dest2','res_dest2')


[2020-11-03 10:10:38,205] [    INFO] - Installing deeplabv3p_xception65_humanseg module
[2020-11-03 10:10:38,430] [    INFO] - Module deeplabv3p_xception65_humanseg already installed in /home/aistudio/.paddlehub/modules/deeplabv3p_xception65_humanseg
[2020-11-03 10:11:13,063] [    INFO] - Installing deeplabv3p_xception65_humanseg module
[2020-11-03 10:11:13,065] [    INFO] - Module deeplabv3p_xception65_humanseg already installed in /home/aistudio/.paddlehub/modules/deeplabv3p_xception65_humanseg


抠图完成
抠图完成


## 三、图像合成

将抠出的人物图片合成在想要的背景图片当中。


In [8]:
#将扣出的人像与原图合并
def blend_images(fore_path, base_path ,savepath):
    """
    将抠出的人物图像换背景
    fore_image: 前景图片，抠出的人物图片
    base_image: 背景图片
    """
    filename_list = os.listdir(fore_path)
    for fname in filename_list:
        fore_image = fore_path + '/' + fname 
        base_image = base_path + '/' + fname

    # 读入图片
        base_image = Image.open(base_image).convert('RGB')
        fore_image = Image.open(fore_image).resize(base_image.size)

    # 图片加权合成
        scope_map = np.array(fore_image)[:,:,-1] / 255
        scope_map = scope_map[:,:,np.newaxis]
        scope_map = np.repeat(scope_map, repeats=3, axis=2)
        res_image = np.multiply(scope_map, np.array(fore_image)[:,:,:3]) + np.multiply((1-scope_map), np.array(base_image))
    
    #保存图片
        res_image = Image.fromarray(np.uint8(res_image))
        res_image.save(savepath + '/' + fname)

savepath = 'result_picture'
if os.path.exists(savepath):
    shutil.rmtree(savepath)
os.mkdir(savepath) 
blend_images('res_dest1', 'video_pic', savepath)
blend_images('res_dest2', 'result_picture', savepath)

#删除多余文件以免混淆
shutil.rmtree('dest1')
shutil.rmtree('dest2')
shutil.rmtree('res_dest1')
shutil.rmtree('res_dest2')
print("图片成功合成")

图片成功合成


## 四、图片转视频+音乐




In [9]:
#图片变视频加配乐
def merge_to_mp4():
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video= cv2.VideoCapture('./xxm.mp4')
    fps = video.get(cv2.CAP_PROP_FPS)
    img = Image.open('./result_picture/1.jpg')#需要整合成视频的图片路径
    out = cv2.VideoWriter('result.mp4', fourcc, fps, (img.size[0], img.size[1]))
    files = os.listdir('./result_picture')
    files.sort(key=lambda x: int(x[:-4]))
    for i in files:
        img = cv2.imread('./result_picture/'+i)
        out.write(img)#保存帧
    out.release()
    print('ok!')

def add_audio():
    video_origin = VideoFileClip('xxm.mp4')
    audio=video_origin.audio
    video_result = VideoFileClip('./result.mp4')
    video_result = video_result.set_audio(audio)
    video_result.write_videofile('成品.mp4')
    
    print('ok!')

merge_to_mp4()
add_audio()

ok!
Moviepy - Building video 成品.mp4.
MoviePy - Writing audio in 成品TEMP_MPY_wvf_snd.mp3


                                                                   
t:   0%|          | 0/309 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video 成品.mp4



t: 100%|█████████▉| 308/309 [00:06<00:00, 42.18it/s, now=None]2020-11-03 10:13:34,156-WARNING: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/moviepy/video/io/ffmpeg_reader.py:130: UserWarning: Warning: in file ./result.mp4, 2764800 bytes wanted but 0 bytes read,at frame 308/309, at time 10.27/10.27 sec. Using the last valid frame instead.
  UserWarning)



Moviepy - Done !
Moviepy - video ready 成品.mp4
ok!


## 本人介绍
 AI爱好者一枚，欢迎互相关注：
 https://aistudio.baidu.com/aistudio/personalcenter/thirdview/329995